In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import time
from google.cloud import bigquery
import os

import pandas_gbq
%load_ext google.cloud.bigquery

# Set your default project here
pandas_gbq.context.project = 'snappy-boulder-378707'
pandas_gbq.context.dialect = 'standard'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../letsgo-snappy-boulder-378707-4b7d46801fd1.json"

# Construct a BigQuery client object.
client = bigquery.Client()

# Insert Album info

In [ ]:
%%bigquery
INSERT INTO snappy-boulder-378707.TrackClearInfo.AlbumInfo SELECT id,name,total_tracks,release_date,extract_date, available_markets FROM snappy-boulder-378707.NewReleases.NewAlbums

# Insert track info

In [ ]:
%%bigquery
INSERT INTO snappy-boulder-378707.TrackClearInfo.TrackClearInfo 
SELECT id, name, album_id, artist_id, popularity, explicit, available_markets,extract_date
FROM snappy-boulder-378707.NewReleases.NewTracks

# Insert audio features

In [ ]:
%%bigquery
INSERT INTO snappy-boulder-378707.AudioFeatures.AudioFeatures 
SELECT danceability, energy,key,loudness,mode,speechiness,
acousticness,instrumentalness,liveness,valence,	
tempo,id, duration_ms,time_signature FROM snappy-boulder-378707.NewReleases.NewAudioFeatures

# Statistics about popularity group by genre

In [ ]:
%%bigquery popularity
SELECT id, popularity, extract_date FROM snappy-boulder-378707.TrackClearInfo.TrackClearInfo


In [ ]:
%%bigquery genre
SELECT * FROM snappy-boulder-378707.TrackGenre.Trackgenre

In [ ]:
genre = genre.melt(id_vars=['track_id'], var_name='genre', value_name='value')
genre = genre[genre.value == 1]
genre.drop(['value'], axis=1, inplace=True)

In [ ]:
genre_popularity = pd.merge(popularity, genre, left_on = "id", right_on = "track_id", how="left")

In [ ]:
genre_popularity

In [ ]:
stats = genre_popularity.groupby(['genre','extract_date']).aggregate({'popularity':'sum','id':'count'}).reset_index()

In [ ]:
stats.columns = ['genre','extract_date','popularity','total_tracks']

## creat temp table

In [ ]:
schema = [
    bigquery.SchemaField("genre","STRING"),
    bigquery.SchemaField("extract_date","DATE"),
    bigquery.SchemaField("popularity","INTEGER"),
    bigquery.SchemaField("total_tracks","INTEGER")
]


# TODO(developer): Set table_id to the ID of the table to create.
table_id = "snappy-boulder-378707.GenrePopularity.GenrePopularityMonth"
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, timeout=30)  # Make an API request.

In [15]:
job_config = bigquery.LoadJobConfig(
    schema = schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

job = client.load_table_from_dataframe(stats, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 18 rows and 4 columns to snappy-boulder-378707.GenrePopularity.GenrePopularityMonth


In [18]:
%%bigquery
INSERT INTO snappy-boulder-378707.GenrePopularity.GenrePopularity
SELECT * FROM snappy-boulder-378707.GenrePopularity.GenrePopularityMonth

Query is running:   0%|          |

""


In [19]:
%%bigquery
DROP TABLE snappy-boulder-378707.GenrePopularity.GenrePopularityMonth

Query is running:   0%|          |

""
